In [120]:
import polars as pl
import spacy
import os
from sklearn.feature_extraction.text import TfidfVectorizer

pl.Config(tbl_width_chars=1000)

In [130]:
DATASET = f"{os.getcwd()}/data/dataset_base.parquet"
DATASET

'/Users/diegolopes/repositories/geo-nlq-to-sql/data/dataset_base.parquet'

In [136]:
df = pl.scan_parquet(DATASET)
df

In [137]:
df.head().collect()

id,question,territorial_division,level,geospatial_functions,sql_code
i64,str,str,str,str,str
1,"""Quais escolas urbanas ficam em…","""País""","""Facíl""","""ST_Intersects""","""SELECT e.cd_entidade, m.cd…"
2,"""Quais bibliotecas com CEP ause…","""País""","""Facíl""","""ST_Intersects""","""SELECT b.cd_bib, b.nm_bib,…"
3,"""Quais CRAS dos estados SP e PR…","""País""","""Facíl""","""ST_Intersects""","""SELECT c.nu_identif, c.nm_…"
4,"""Quais CREAS com CEP preenchido…","""País""","""Facíl""","""ST_Intersects""","""SELECT r.nu_identif, r.nm_…"
5,"""Relacione grandes regiões e se…","""País""","""Facíl""","""ST_Intersects""","""SELECT r.cd_regia, r.nm_re…"


In [129]:
df.write_parquet(f"{os.getcwd()}/data/dataset_base.parquet")

In [63]:
nlp = spacy.load('pt_core_news_lg')

In [65]:
documents = []

for row in df.iter_rows():
    documents.append(nlp(row[1]))

In [89]:
corpus = df['question'].to_list()
vectorizer = TfidfVectorizer(stop_words=['?', '.', '!'])
X = vectorizer.fit_transform(corpus)

In [91]:
list(vectorizer.get_stop_words())

['!', '?', '.']

In [116]:
vectorizer.vocabulary_['urbanas']

1158

In [119]:
feature_names = vectorizer.get_feature_names_out()
feature_names[1158]

'urbanas'

In [117]:
X[0, 1158]

np.float64(0.36136261353612775)

In [113]:
doc = 0
feature_index = X[doc,:].nonzero()[1]
tfidf_scores = zip(feature_index, [X[doc, x] for x in feature_index])

In [114]:
tfidf_scores

In [115]:
for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
  print(f'{w} - {s}')

quais - 0.15604167824789855
escolas - 0.10557360964668408
urbanas - 0.36136261353612775
ficam - 0.21079411387058986
em - 0.13792899320918345
municípios - 0.24803302700837976
com - 0.13657820733020737
área - 0.20147902818623423
000 - 0.32849464610891094
km2 - 0.36433207330096
nome - 0.29595500667806557
iniciando - 0.4797054460348449
por - 0.16182313396680487
são - 0.2552565684608787
